In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [1]:
import pandas as pd
import numpy as np
import tensorflow
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU, Activation, Dropout, Dense, Input, Bidirectional, Layer
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop

from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_score, recall_score
import pickle
from tensorflow import keras
import keras.backend as kb

from tensorboard.plugins.hparams import api as hp


In [2]:
def creazione_modello_GloVe(filename):
    f = open(filename, encoding="utf8")
    embeding_index = {}
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeding_index[word] = coefs
    f.close()
    return embeding_index

In [3]:
#dataset = pickle.load(open("drive/MyDrive/ml_project/dataset_SLS.pickle", "rb"))
dataset = pickle.load(open("dataset/dataset_SLS.pickle", "rb"))

In [4]:
Y = np.array(dataset["sentiment"])

In [5]:
x_train,x_test,y_train,y_test = train_test_split(dataset["sentence"],Y, test_size=0.1, shuffle=True)

In [6]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(2700,) (2700,)
(300,) (300,)


In [32]:
print("numero di positivi nel train: ", list(y_train).count(1))
print("numero di negativi nel train: ", list(y_train).count(0))

print("numero di positivi nel test: ", list(y_test).count(1))
print("numero di negativi nel test: ", list(y_test).count(0))

numero di positivi nel train:  1351
numero di negativi nel train:  1349
numero di positivi nel test:  149
numero di negativi nel test:  151


In [7]:
embedding = creazione_modello_GloVe("dataset/glove.6B.50d.txt")

In [8]:
tokenizer = Tokenizer(num_words=140000)
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index

In [9]:
vocab_len = len(word_index)+1
embedding_vector_len = embedding["banana"].shape[0]
embedding_matrix = np.zeros((vocab_len, embedding_vector_len))

for word, index in word_index.items():
    vector = embedding.get(word)
    if vector is not None:
        embedding_matrix[index, :] = vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embedding_vector_len, input_length=300, weights=[embedding_matrix])

In [10]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=kb.squeeze(kb.tanh(kb.dot(x,self.W)+self.b),axis=-1)
        at=kb.softmax(et)
        at=kb.expand_dims(at,axis=-1)
        output=x*at
        return kb.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [11]:
x_train_index = tokenizer.texts_to_sequences(x_train)
x_train_index = pad_sequences(x_train_index, maxlen=300)

In [12]:
y_train_categorical = keras.utils.to_categorical(y_train, 2)
y_test_categorical = keras.utils.to_categorical(y_test, 2)

In [13]:
x_test_index = tokenizer.texts_to_sequences(x_test)
x_test_index = pad_sequences(x_test_index, maxlen=300)

In [37]:
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(["adam", "rmsprop"]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1,0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32, 64]))

log_dir = 'drive/MyDrive/ml_project/SLS/logs/SLS_BiGRUAtt_HP_complete'
METRIC_ACCURACY = 'accuracy'
METRIC_PRECISION = "precision"
METRIC_RECALL = "recall"

with tensorflow.summary.create_file_writer(log_dir).as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name="Accuracy"), hp.Metric(METRIC_PRECISION, display_name="Precision"), hp.Metric(METRIC_RECALL, display_name="Recall")],
    )

In [44]:
def train_test_model(hparams):
    model = Sequential([
        embedding_layer,
        Bidirectional(GRU(units=hparams[HP_NUM_UNITS], return_sequences=True, dropout=hparams[HP_DROPOUT])),
        Dense(hparams[HP_NUM_UNITS]),
        Dense(hparams[HP_NUM_UNITS]),
        attention(),
        Dense(2, activation="sigmoid"),
    ])
    model.compile(optimizer=hparams[HP_OPTIMIZER], loss="binary_crossentropy", metrics=["accuracy"])
    model.fit(x_train_index[:2100], y_train_categorical[:2100], epochs=10, batch_size=128, verbose=1, validation_data=(x_train_index[2100:], y_train_categorical[2100:]), callbacks=[tensorflow.keras.callbacks.TensorBoard(log_dir), hp.KerasCallback(log_dir, hparams)])
    _, accuracy = model.evaluate(x_test_index, y_test_categorical)
    y_pred = np.argmax(model.predict(x_test_index), axis=1).astype("float32")
    print(classification_report(y_test, y_pred))
    model.save("drive/MyDrive/ml_project/SLS/models/SLS_BiGRUAtt_HP_complete/{}_{}_{}".format(hparams[HP_NUM_UNITS], hparams[HP_DROPOUT], hparams[HP_OPTIMIZER]))
    return accuracy, precision_score(y_test, y_pred), recall_score(y_test, y_pred)

In [42]:
def run(run_dir, hparams):
    with tensorflow.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy, precision, recall = train_test_model(hparams)
        tensorflow.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        tensorflow.summary.scalar(METRIC_PRECISION, precision, step=1)
        tensorflow.summary.scalar(METRIC_RECALL, recall, step=1)

In [45]:
n_esecuzione = 0
for num_units in HP_NUM_UNITS.domain.values:
        for dropout in HP_DROPOUT.domain.values:
            for opt in HP_OPTIMIZER.domain.values:
                hparams = {
                    HP_DROPOUT: dropout,
                    HP_NUM_UNITS: num_units,
                    HP_OPTIMIZER: opt,
                    }
                print("esecuzione: ", n_esecuzione)
                print({h.name: hparams[h] for h in hparams})
                run(log_dir + "/esecuzione_{}".format(n_esecuzione), hparams)
                n_esecuzione += 1 

esecuzione:  0
{'dropout': 0.1, 'num_units': 16, 'optimizer': 'adam'}
Epoch 1/10
17/17 [==============================] - 11s 426ms/step - loss: 0.6938 - accuracy: 0.4848 - val_loss: 0.6924 - val_accuracy: 0.5183
Epoch 2/10
17/17 [==============================] - 6s 345ms/step - loss: 0.6908 - accuracy: 0.5790 - val_loss: 0.6897 - val_accuracy: 0.6083
Epoch 3/10
17/17 [==============================] - 6s 361ms/step - loss: 0.6829 - accuracy: 0.6271 - val_loss: 0.6791 - val_accuracy: 0.6233
Epoch 4/10
17/17 [==============================] - 6s 341ms/step - loss: 0.6480 - accuracy: 0.6881 - val_loss: 0.6379 - val_accuracy: 0.6300
Epoch 5/10
17/17 [==============================] - 6s 338ms/step - loss: 0.5515 - accuracy: 0.7438 - val_loss: 0.5485 - val_accuracy: 0.7267
Epoch 6/10
17/17 [==============================] - 6s 362ms/step - loss: 0.3637 - accuracy: 0.8624 - val_loss: 0.5251 - val_accuracy: 0.8117
Epoch 7/10
17/17 [==============================] - 6s 365ms/step - loss: 0.2

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/SLS/models/SLS_BiGRUAtt_HP_complete/16_0.1_adam/assets


INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/SLS/models/SLS_BiGRUAtt_HP_complete/16_0.1_adam/assets


esecuzione:  1
{'dropout': 0.1, 'num_units': 16, 'optimizer': 'rmsprop'}
Epoch 1/10
17/17 [==============================] - 13s 468ms/step - loss: 0.6878 - accuracy: 0.5600 - val_loss: 0.6841 - val_accuracy: 0.6650
Epoch 2/10
17/17 [==============================] - 6s 370ms/step - loss: 0.6654 - accuracy: 0.6790 - val_loss: 0.6591 - val_accuracy: 0.6433
Epoch 3/10
17/17 [==============================] - 6s 367ms/step - loss: 0.5973 - accuracy: 0.7410 - val_loss: 0.6059 - val_accuracy: 0.6817
Epoch 4/10
17/17 [==============================] - 6s 343ms/step - loss: 0.4687 - accuracy: 0.8543 - val_loss: 0.5440 - val_accuracy: 0.7017
Epoch 5/10
17/17 [==============================] - 6s 344ms/step - loss: 0.3241 - accuracy: 0.9152 - val_loss: 0.4459 - val_accuracy: 0.8117
Epoch 6/10
17/17 [==============================] - 6s 347ms/step - loss: 0.2109 - accuracy: 0.9443 - val_loss: 0.4393 - val_accuracy: 0.8317
Epoch 7/10
17/17 [==============================] - 6s 340ms/step - loss: 

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/SLS/models/SLS_BiGRUAtt_HP_complete/16_0.1_rmsprop/assets


...

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/SLS/models/SLS_BiGRUAtt_HP_complete/64_0.9_adam/assets


INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/SLS/models/SLS_BiGRUAtt_HP_complete/64_0.9_adam/assets


esecuzione:  53
{'dropout': 0.9, 'num_units': 64, 'optimizer': 'rmsprop'}
Epoch 1/10
17/17 [==============================] - 19s 854ms/step - loss: 0.6441 - accuracy: 0.6629 - val_loss: 0.5048 - val_accuracy: 0.7983
Epoch 2/10
17/17 [==============================] - 12s 736ms/step - loss: 0.3191 - accuracy: 0.9095 - val_loss: 0.5571 - val_accuracy: 0.7950
Epoch 3/10
17/17 [==============================] - 13s 764ms/step - loss: 0.1509 - accuracy: 0.9533 - val_loss: 0.7297 - val_accuracy: 0.8083
Epoch 4/10
17/17 [==============================] - 13s 761ms/step - loss: 0.1084 - accuracy: 0.9605 - val_loss: 0.9365 - val_accuracy: 0.8150
Epoch 5/10
17/17 [==============================] - 13s 763ms/step - loss: 0.0786 - accuracy: 0.9738 - val_loss: 1.0785 - val_accuracy: 0.8167
Epoch 6/10
17/17 [==============================] - 13s 757ms/step - loss: 0.0694 - accuracy: 0.9748 - val_loss: 1.1134 - val_accuracy: 0.8183
Epoch 7/10
17/17 [==============================] - 13s 763ms/step -

INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/SLS/models/SLS_BiGRUAtt_HP_complete/64_0.9_rmsprop/assets


INFO:tensorflow:Assets written to: drive/MyDrive/ml_project/SLS/models/SLS_BiGRUAtt_HP_complete/64_0.9_rmsprop/assets


In [46]:
x_train_file = open("drive/MyDrive/ml_project/SLS/x_train.pickle", "wb")
pickle.dump(x_train, x_train_file)
x_train_file.close()


x_test_file = open("drive/MyDrive/ml_project/SLS/x_test.pickle", "wb")
pickle.dump(x_test, x_test_file)
x_test_file.close()


y_train_file = open("drive/MyDrive/ml_project/SLS/y_train.pickle", "wb")
pickle.dump(y_train, y_train_file)
y_train_file.close()

y_test_file = open("drive/MyDrive/ml_project/SLS/y_test.pickle", "wb")
pickle.dump(y_test, y_test_file)
y_test_file.close()

Dopo aver effettuato  diverse esecuzioni, la migliore configurazione risulta essere: NUM_UNITS:16, Dropout:0.4, OPT: RMSProp con accuracy del 84% dunque al variare di tali parametri non si riesce ad ottenere un risultato migliore mostrato dall'articolo (86%). L'ultima prova che si effettua è quella di modificare il learning rate

In [14]:
HP_LR = hp.HParam('learning_rate', hp.Discrete([0.0001, 0.001, 0.005, 0.009, 0.1, 0.2, 0.25, 0.3]))

#log_dir = 'drive/MyDrive/ml_project/SLS/logs/SLS_BiGRUAtt_HP_complete_LR'
log_dir = 'logs/SLS_BiGRUAtt_HP_complete_LR'
METRIC_ACCURACY = 'accuracy'
METRIC_PRECISION = "precision"
METRIC_RECALL = "recall"

with tensorflow.summary.create_file_writer(log_dir).as_default():
    hp.hparams_config(
        hparams=[HP_LR],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name="Accuracy"), hp.Metric(METRIC_PRECISION, display_name="Precision"), hp.Metric(METRIC_RECALL, display_name="Recall")],
    )

In [18]:
def train_test_model(hparams):
    model = Sequential([
        embedding_layer,
        Bidirectional(GRU(units=16, return_sequences=True, dropout=0.4)),
        Dense(16),
        Dense(16),
        attention(),
        Dense(2, activation="sigmoid"),
    ])
    opt = RMSprop(learning_rate=hparams[HP_LR])
    model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])
    model.fit(x_train_index[:2100], y_train_categorical[:2100], epochs=10, batch_size=128, verbose=1, validation_data=(x_train_index[2100:], y_train_categorical[2100:]), callbacks=[tensorflow.keras.callbacks.TensorBoard(log_dir), hp.KerasCallback(log_dir, hparams)])
    _, accuracy = model.evaluate(x_test_index, y_test_categorical)
    y_pred = np.argmax(model.predict(x_test_index), axis=1).astype("float32")
    print(classification_report(y_test, y_pred))
    model.save("models/BiGRUAtt_HP_complete_LR/{}_{}_{}".format(16, 0.4, hparams[HP_LR]))
    return accuracy, precision_score(y_test, y_pred), recall_score(y_test, y_pred)

In [16]:
def run(run_dir, hparams):
    with tensorflow.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy, precision, recall = train_test_model(hparams)
        tensorflow.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        tensorflow.summary.scalar(METRIC_PRECISION, precision, step=1)
        tensorflow.summary.scalar(METRIC_RECALL, recall, step=1)

In [19]:
n_esecuzione = 0
for lr in HP_LR.domain.values:
    hparams = {
        HP_LR: lr,
        }
    print("esecuzione: ", n_esecuzione)
    print({h.name: hparams[h] for h in hparams})
    run(log_dir + "/esecuzione_{}".format(n_esecuzione), hparams)
    n_esecuzione += 1 

esecuzione:  0
{'learning_rate': 0.0001}
Epoch 1/10
17/17 [==============================] - 9s 362ms/step - loss: 0.6933 - accuracy: 0.5033 - val_loss: 0.6930 - val_accuracy: 0.5050
Epoch 2/10
17/17 [==============================] - 5s 296ms/step - loss: 0.6931 - accuracy: 0.4986 - val_loss: 0.6929 - val_accuracy: 0.5217
Epoch 3/10
17/17 [==============================] - 5s 279ms/step - loss: 0.6931 - accuracy: 0.5005 - val_loss: 0.6929 - val_accuracy: 0.5283
Epoch 4/10
17/17 [==============================] - 4s 253ms/step - loss: 0.6929 - accuracy: 0.5310 - val_loss: 0.6929 - val_accuracy: 0.5083
Epoch 5/10
17/17 [==============================] - 4s 245ms/step - loss: 0.6928 - accuracy: 0.5229 - val_loss: 0.6930 - val_accuracy: 0.4983
Epoch 6/10
17/17 [==============================] - 4s 263ms/step - loss: 0.6926 - accuracy: 0.5167 - val_loss: 0.6929 - val_accuracy: 0.4883
Epoch 7/10
17/17 [==============================] - 4s 242ms/step - loss: 0.6926 - accuracy: 0.5181 - val_l